In [1]:
import eccodes
import pandas as pd
import os

In [2]:
def list_grib_variables(grib_path):
    variables = []
    
    if not os.path.exists(grib_path):
        raise FileNotFoundError(f"GRIB 文件未找到: {grib_path}")
    
    with open(grib_path, 'rb') as f:
        while True:
            gid = eccodes.codes_grib_new_from_file(f)
            if gid is None:
                break  
            
            try:
                var_code = eccodes.codes_get(gid, 'indicatorOfParameter')
                var_abbr = eccodes.codes_get(gid, 'shortName')
                var_desc = eccodes.codes_get(gid, 'name')
                var_units = eccodes.codes_get(gid, 'units')
                level_type = eccodes.codes_get(gid, 'indicatorOfTypeOfLevel')
                level = eccodes.codes_get(gid, 'level')
                tri = eccodes.codes_get(gid, 'timeRangeIndicator')
                
                
                variables.append({
                    'Code': var_code,
                    'Short_name': var_abbr,
                    'Description': var_desc,
                    'Units': var_units,
                    'Leveltype': level_type,
                    'Level': level,
                    'TRI': tri,
                })
            except Exception as e:
                print(f"Error processing message: {e}")
            finally:
                eccodes.codes_release(gid)
    
    # transform to Pandas DataFrame
    df = pd.DataFrame(variables)
    
    # drop the duplicates
    df_unique = df.drop_duplicates()
    
    return df_unique

In [3]:
grib_first_path = "HARMONIE_AROME_meteo_24hrs/mbr000/fc2024070912+001GB_UWCW01_N20e"  # change to your path

variables_df = list_grib_variables(grib_first_path)
variables_df

,Code,Short_name,Description,Units,Leveltype,Level,TRI
0,1,unknown,unknown,unknown,103,0,0
1,6,unknown,unknown,unknown,pl,300,0
2,11,unknown,unknown,unknown,pl,850,0
3,11,unknown,unknown,unknown,sfc,0,0
4,11,unknown,unknown,unknown,sfc,2,0
5,20,unknown,unknown,unknown,sfc,0,0
6,33,unknown,unknown,unknown,pl,700,0
7,34,unknown,unknown,unknown,pl,700,0
8,33,unknown,unknown,unknown,sfc,10,0
9,34,unknown,unknown,unknown,sfc,10,0


In [4]:
HARMONIE_PARAMS = {
    1: {'shortName': ['PMSL', 'PSRF'], 'name': ['Pressure altitude above MSL', 'Pressure height above ground'], 'units': 'Pa'},
    6: {'shortName': 'GEOP', 'name': 'Geopotential', 'units': 'm²/s²'},
    11: {'shortName': 'TMP', 'name': 'Temperature', 'units': 'K'},
    17: {'shortName': '2TD', 'name': 'Dew-point temperature 2m', 'units': 'K'},
    20: {'shortName': 'VIS', 'name': 'Visibility', 'units': 'm'},
    33: {'shortName': 'UGRD', 'name': 'U-component of wind', 'units': 'm/s'},
    34: {'shortName': 'VGRD', 'name': 'V-component of wind', 'units': 'm/s'},
    40: {'shortName': 'VVEL', 'name': 'Vertical velocity', 'units': 'm/s'},
    41: {'shortName': 'ABSV', 'name': 'Absolute vorticity', 'units': 's⁻¹'},
    52: {'shortName': 'RH', 'name': 'Relative humidity', 'units': '%'},
    65: {'shortName': 'WEASD', 'name': 'Water equivalent of snow', 'units': 'kg/m²'},
    71: {'shortName': 'TCC', 'name': 'Total cloud cover', 'units': '%'},
    73: {'shortName': 'LCC', 'name': 'Low cloud cover', 'units': '%'},
    74: {'shortName': 'MCC', 'name': 'Medium cloud cover', 'units': '%'},
    75: {'shortName': 'HCC', 'name': 'High cloud cover', 'units': '%'},
    115: {'shortName': 'SWR', 'name': 'Net shortwave radiation', 'units': 'J/m²'},
    116: {'shortName': 'LWR', 'name': 'Net longwave radiation', 'units': 'J/m²'},
    117: {'shortName': 'GRAD', 'name': 'Global radiation flux', 'units': 'W/m²'},
    121: {'shortName': 'MX2T', 'name': 'Maximum temperature 2m', 'units': 'K'},
    122: {'shortName': 'MN2T', 'name': 'Minimum temperature 2m', 'units': 'K'},
    160: {'shortName': 'CAPE', 'name': 'Convective available potential energy', 'units': 'J/kg'},
    161: {'shortName': 'CIN', 'name': 'Convective inhibition', 'units': 'J/kg'},
    162: {'shortName': 'CSULF', 'name': 'U-momentum of gusts', 'units': 'm/s'},
    163: {'shortName': 'CSDLF', 'name': 'V-momentum of gusts', 'units': 'm/s'},
    165: {'shortName': 'BLH', 'name': 'Boundary layer height', 'units': 'm'},
    181: {'shortName': 'LPSX', 'name': 'Rain', 'units': 'kg/m²'},
    184: {'shortName': 'HGTY', 'name': 'Snow', 'units': 'kg/m²'},
    186: {'shortName': 'ICNG', 'name': 'Cloud base', 'units': 'm'},
    187: {'shortName': 'CEIL', 'name': 'Cloud ceiling', 'units': 'm'},
    201: {'shortName': 'ICWAT', 'name': 'Graupel', 'units': 'kg/m²'},
    209: {'shortName': 'TWATER', 'name': 'Total column water', 'units': 'kg/m²'}
}

def list_grib_variables(grib_path):
    variables = []
    
    if not os.path.exists(grib_path):
        raise FileNotFoundError(f"GRIB 文件未找到: {grib_path}")
    
    with open(grib_path, 'rb') as f:
        while True:
            gid = eccodes.codes_grib_new_from_file(f)
            if gid is None:
                break
            
            try:
                var_code = eccodes.codes_get(gid, 'indicatorOfParameter')
                level_type = eccodes.codes_get(gid, 'indicatorOfTypeOfLevel')
                level = eccodes.codes_get(gid, 'level')
                tri = eccodes.codes_get(gid, 'timeRangeIndicator')
                
                # get parameter info
                param_info = HARMONIE_PARAMS.get(var_code, {
                    'shortName': 'unknown',
                    'name': 'unknown',
                    'units': 'unknown'
                })
                
                short_name = param_info['shortName']
                if isinstance(short_name, list):
                    short_name = short_name[1] if level_type in ['105', 'sfc'] else short_name[0]
                
                name = param_info['name']
                if isinstance(name, list):
                    name = name[1] if level_type in ['105', 'sfc'] else name[0]
                
                # standarize level_type
                if level_type == '100':
                    level_type = 'pl'
                elif level_type in ['103', '105']:
                    level_type = 'sfc'
                
                variables.append({
                    'Code': var_code,
                    'Short_name': short_name,
                    'Description': name,
                    'Units': param_info['units'],
                    'Leveltype': level_type,
                    'Level': level,
                    'TRI': tri
                })
                
            except Exception as e:
                print(f"Error processing message: {e}")
            finally:
                eccodes.codes_release(gid)
    
    df = pd.DataFrame(variables)
    df_unique = df.drop_duplicates()
    
    return df_unique

# test
grib_path = "HARMONIE_AROME_meteo_24hrs/mbr000/fc2024070912+001GB_UWCW01_N20e" # change to your path
variables_df = list_grib_variables(grib_path)
variables_df = variables_df.sort_values('Code')

print(variables_df)

    Code Short_name                            Description  Units Leveltype  \
0      1       PMSL            Pressure altitude above MSL     Pa       sfc   
1      6       GEOP                           Geopotential  m²/s²        pl   
2     11        TMP                            Temperature      K        pl   
3     11        TMP                            Temperature      K       sfc   
4     11        TMP                            Temperature      K       sfc   
5     20        VIS                             Visibility      m       sfc   
6     33       UGRD                    U-component of wind    m/s        pl   
8     33       UGRD                    U-component of wind    m/s       sfc   
14    33       UGRD                    U-component of wind    m/s       sfc   
10    33       UGRD                    U-component of wind    m/s       sfc   
12    33       UGRD                    U-component of wind    m/s       sfc   
7     34       VGRD                    V-component o